# Importing relevant dependencies

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

# Pandas display options:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max.columns', None)
#pd.set_option('display.max.rows', None)

# Loading in the raw data

In [2]:
camper = pd.read_csv('/Users/darkxiii/Docs/IE Data Science Bootcamp/Camper Project/Database/Consumidor_Venta_Producto_IE_2018-2020_v3.csv', parse_dates=['FECHA_FACTURA'])

In [3]:
# Base de datos original:
camper.head()

,FACTURA_ID,FACTURA_POSICION_ID,CUSTOMER_ID,FACTURA_CLASE_DOCUMENTO_ID,ANO_MES_FACTURA,ANO_FACTURA,MES_FACTURA,FECHA_FACTURA,IMP_VENTA_NETO_EUR,CANAL_VENTA_ID,CANAL_VENTA_DESC,TEMPORADA_COMERCIAL_ID,TEMPORADA_COMERCIAL_DESC,PRODUCTO_ID,TALLA,MATERIAL_ID,NUMERO_DEUDOR_PAIS_ID,NUMERO_DEUDOR_PAIS_DESC,VENTA_DEVOLUCION,JERARQUIA_PROD_ID,GRUPO_ARTICULO_PRODUCTO_ID,GRUPO_ARTICULO,CONCEPTO,LINEA,GENERO_PRODUCTO,CATEGORIA,TIPOLOGIA,COLOR,CONSUMER_COLOR,CREMALLERA,CORDONES,OUTSOLE_SUELA_TIPO,OUTSOLE_SUELA_SUBTIPO,PLANTILLA_EXTRAIBLE,CONTACTO_SN,EDAD_SN,GENERO_CONTACTO,EDAD_COMPRA,EDAD_RANGO_COMPRA,PAIS_CONTACTO,PAIS_CONTACTO_DESC,CIUDAD_CONTACTO,IDIOMA_CONTACTO
0,2856600006,1,heLvMOhLzUq8Q3JQDKC1VYitTQpF5BpDPxJmUK3eoaQ=,ZTON,201801,2018,1,2018-01-01,155.75,20,Online,85,85 - F/W 2017,18637-001,42,18637-001-42,TR,TURKEY,V,101ATMATMWH18637,1.00,Zapatos Adulto,Atom,Atom Work,MEN,Men Shoe,Blucher,Black,Negro,NO,Con cordones,FLAT,FLAT,No extraible,S,S,MAN,30,30-39,TR,TURKEY,Izmit (Kocaeli),TR
1,2844600022,4,LF/nHsMbdxZfH9ymPjCNGMO7v5nwb4kLLoV77YZkqHA=,ZTON,201801,2018,1,2018-01-01,112.07,20,Online,85,85 - F/W 2017,K100243-006,41,K100243-006-41,US,UNITED STATES,V,101TRNTRN0HK100243,1.00,Zapatos Adulto,Truman,Truman,MEN,Men Shoe,Blucher,Black,Negro,NO,Con cordones,FLAT,FLAT,No extraible,S,S,NV,42,40-49,US,UNITED STATES,Jersey City,EN
2,2844600022,1,LF/nHsMbdxZfH9ymPjCNGMO7v5nwb4kLLoV77YZkqHA=,ZTON,201801,2018,1,2018-01-01,184.27,20,Online,85,85 - F/W 2017,16002-204,41,16002-204-41,US,UNITED STATES,V,101PELPELAH16002,1.00,Zapatos Adulto,Pelotas,Pelotas Ariel,MEN,Men Shoe,Oxford,Dark Brown,Marrón,NO,Con cordones,FLAT,FLAT,Extraible,S,S,NV,42,40-49,US,UNITED STATES,Jersey City,EN
3,2844600029,2,dZTljcECKzzhrFvudkxLjJGZOHMioStfcX351srHZrk=,ZTON,201801,2018,1,2018-01-01,169.19,20,Online,85,85 - F/W 2017,46829-036,40,46829-036-40,US,UNITED STATES,V,101PEUPEUPM46829,1.00,Adult Shoes,Peu,Peu Pista,WOMEN,Women Ankle Boot,Lace Up Bootie,Black,Black,NO,With laces,FLAT,FLAT,Extractable,S,S,WOMAN,46,40-49,US,UNITED STATES,Mesa,EN
4,2844600014,2,Xb5/e2P6hITup7gs5k6G2jjFC/7Tahyp5V4txDKbzFU=,ZTON,201801,2018,1,2018-01-01,132.55,20,Online,85,85 - F/W 2017,K300192-001,44,K300192-001-44,US,UNITED STATES,V,101PEUPEUCHK300192,1.00,Zapatos Adulto,Peu,Peu Cami,MEN,Men Ankle Boot,Basket bootie,Black,Negro,SI,Con cordones,FLAT,FLAT,Extraible,S,S,NV,49,40-49,US,UNITED STATES,Stafford,EN


# Data Preprocessing

In [4]:
# Copia de la base de datos original para simplificado y limpieza
camperlean = camper.copy(deep = True)

In [5]:
# ========================================== FILTRADO INICIAL ===========================================

# Filtrando base de datos para quedarnos SOLO con jerarquía de productos 1 (CAMPER):
camperlean['JERARQUIA_1'] = list(camperlean['JERARQUIA_PROD_ID'].str.startswith('1'))
camperlean = camperlean[(camperlean.JERARQUIA_1 == True)]

# Unificando GRUPO_ARTICULO:
camperlean['GRUPO_ARTICULO'] = camperlean['GRUPO_ARTICULO'].replace(['Adult Shoes'],'Zapatos Adulto')
camperlean['GRUPO_ARTICULO'] = camperlean['GRUPO_ARTICULO'].replace(["Kid's Shoes"],'Kids')
# Filtrando base de datos para quedarnos SOLO con zapatos (niños y adultos):
camperlean = camperlean[(camperlean.GRUPO_ARTICULO == 'Zapatos Adulto') | (camperlean.GRUPO_ARTICULO == 'Kids')]

# Filtrando base de datos para quedarnos SOLO con ventas:
camperlean = camperlean[(camperlean.FACTURA_CLASE_DOCUMENTO_ID == 'ZTON')]

# Filtrado de base de datos para limpieza de variable CATEGORIA (categorias obsoletas, errores, etc.) 
camperlean = camperlean.loc[~camperlean.CATEGORIA.isin(['83', '80', 'FW - Shoe', 'Kids - Closed Sandal', 'Boy - Shoe', 'Kids - High Boot', 'Boy - Sneaker-Bootie', 'Girl - Ankle-boot', 'Kids - High Boot'])]


# =================================== CONVERSIÓN DE DATOS VACIOS A NaN ==================================

# Conversión a NaN de los 0 de EDAD_COMPRA:
camperlean.EDAD_COMPRA.replace(0, np.nan, inplace=True)

# Conversión a NaN de los NS de EDAD_RANGO_COMPRA:
camperlean.EDAD_RANGO_COMPRA.replace('NS', np.nan, inplace=True)


# ============================================ MANEJO DE NaNs ===========================================

# -------- ELIMINADO DE FILAS CON NaNs ---------
# Eliminado de NaNs de TALLA:
camperlean.dropna(subset=['TALLA'], inplace=True)

# Eliminado de 8058 NaNs de CIUDAD_CONTACTO
camperlean.dropna(subset=['CIUDAD_CONTACTO'], inplace=True)

# Eliminado de 4642 NaNs de TIPOLOGIA
camperlean.dropna(subset=['TIPOLOGIA'], inplace=True)

# Eliminado de 4492 NaNs de CATEGORIA
camperlean.dropna(subset=['CATEGORIA'], inplace=True)

# Eliminado de 3423 NaNs de CONSUMER_COLOR
camperlean.dropna(subset=['CONSUMER_COLOR'], inplace=True)

# Eliminado de 2436 NaNs de IDIOMA_CONTACTO
camperlean.dropna(subset=['IDIOMA_CONTACTO'], inplace=True)

# Eliminado de 1681 NaNs de PAIS_CONTACTO
camperlean.dropna(subset=['PAIS_CONTACTO'], inplace=True)

# Eliminado de 1147 NaNs de CUSTOMER_ID
camperlean.dropna(subset=['CUSTOMER_ID'], inplace=True)

# Eliminado de 888 NaNs de IMP_VENTA_NETO_EUR
camperlean.dropna(subset=['IMP_VENTA_NETO_EUR'], inplace=True)



# -------- EXTRAPOLADO DE DATOS DESDE OTRAS FILAS PARA LLENADO DE NaNs ---------
# Rellenado de NaNs de GENERO_PRODUCTO a partir de JERARQUIA_PROD_ID:
"""def extraer_categoria(celda):
        m = len(celda)
        if m==16:
          # Si la longitud es 16 extraemos el caracter 11
          return celda[10]
        else:
          # Si la longitud es diferente extraemos el caracter 11
          return celda[10]
    
#camperlean['CATEGORIA_INFERIDA'] = camperlean["JERARQUIA_PROD_ID"].map(extraer_categoria)"""

camperlean['CATEGORIA_INFERIDA'] = camperlean.JERARQUIA_PROD_ID.str[10]
camperlean['CATEGORIA_INFERIDA'].replace({"H":"MEN", "M":"WOMEN", "U":"KIDS"}, inplace=True)
camperlean.GENERO_PRODUCTO.fillna(value=camperlean.CATEGORIA_INFERIDA, inplace=True)

# Rellenado de NaNs de COLOR a partir de CONSUMER_COLOR:
camperlean.COLOR.fillna(camperlean.CONSUMER_COLOR , inplace=True)


# -------- REEMPLAZO DE NaNs y Valores Vacios por 'UNDEFINED' ---------
# Conversión a UNDEFINED de los 500k NV y NaNs de GENERO_CONTACTO
camperlean.GENERO_CONTACTO.replace('NV', 'UNDEFINED', inplace=True)
camperlean.GENERO_CONTACTO.replace(np.nan, 'UNDEFINED', inplace=True)

# Conversión a UNDEFINED de los 250k NaNs de EDAD_COMPRA y EDAD_RANGO_COMPRA
camperlean.EDAD_COMPRA.replace(np.nan, 'UNDEFINED', inplace=True)
camperlean.EDAD_RANGO_COMPRA.replace(np.nan, 'UNDEFINED', inplace=True)

# Remplazo de NaNs de Linea 'Together Kremer'
camperlean.loc[(camperlean.LINEA == 'Together Kremer'), 'OUTSOLE_SUELA_TIPO'] = 'FLAT'
camperlean.loc[(camperlean.LINEA == 'Together Kremer'), 'OUTSOLE_SUELA_SUBTIPO'] = 'FLAT'

# Remplazo de NaNs de Linea 'Together Willhelm'
camperlean.loc[(camperlean.LINEA == 'Together Willhelm'), 'OUTSOLE_SUELA_TIPO'] = 'FLAT'
camperlean.loc[(camperlean.LINEA == 'Together Willhelm'), 'OUTSOLE_SUELA_SUBTIPO'] = 'FLAT'

# Remplazo de NaNs de Linea 'Pelotas Together Kva'
camperlean.loc[(camperlean.LINEA == 'Pelotas Together Kva'), 'OUTSOLE_SUELA_TIPO'] = 'FLAT'
camperlean.loc[(camperlean.LINEA == 'Pelotas Together Kva'), 'OUTSOLE_SUELA_SUBTIPO'] = 'FLAT'

# ======================================== CONVERSIÓN DE DATATYPES ======================================

# Transformando FECHA_FACTURA a datetime
camperlean['FECHA_FACTURA'] = camperlean['FECHA_FACTURA'].astype(np.datetime64)

# Transformando TALLA a int:
camperlean['TALLA'] = camperlean['TALLA'].astype(np.int64)


# ======================================== UNIFICACIÓN DE VARIABLES =====================================

# Unificación de variable COLOR a un solo idioma:
camperlean.COLOR.replace("Мульти ассорти", "Multi - Assorted", inplace=True)
camperlean.COLOR.replace("желтый", "Medium Yellow", inplace=True)
camperlean.COLOR.replace("красный", "Medium Red", inplace=True)
camperlean.COLOR.replace("розовый", "Medium Pink", inplace=True)
camperlean.COLOR.replace("Negro","Black",  inplace=True)
camperlean.COLOR.replace("Azul","Blue",  inplace=True)
camperlean.COLOR.replace("Gris", "Grey", inplace=True)
camperlean.COLOR.replace("Marr?n","Brown",  inplace=True)
#camperlean.COLOR.replace("Marr?n","Brown Grey",  inplace=True) ???
camperlean.COLOR.replace("Verde","Green",  inplace=True)
camperlean.COLOR.replace("Rojo","Red",  inplace=True)
camperlean.COLOR.replace("Rosa", "Pink", inplace=True)
camperlean.COLOR.replace("Blanco","White",  inplace=True)
#camperlean.COLOR.replace("Yellow","Nude",  inplace=True) ???
camperlean.COLOR.replace("Amarillo","Yellow",  inplace=True)
camperlean.COLOR.replace("Naranja","Orange",  inplace=True)
camperlean.COLOR.replace("Violeta","Purple",  inplace=True)
camperlean.COLOR.replace("Burdeos","Burgundy",  inplace=True)

# Unificación de variable CONSUMER_COLOR a un solo idioma
camperlean.CONSUMER_COLOR.replace("Black", "Negro", inplace=True)
camperlean.CONSUMER_COLOR.replace("Blue", "Azul", inplace=True)
camperlean.CONSUMER_COLOR.replace("Grey", "Gris", inplace=True)
camperlean.CONSUMER_COLOR.replace("Brown", "Marr?n", inplace=True)
camperlean.CONSUMER_COLOR.replace("Brown Grey", "Marr?n", inplace=True)
camperlean.CONSUMER_COLOR.replace("Green", "Verde", inplace=True)
camperlean.CONSUMER_COLOR.replace("Red", "Rojo", inplace=True)
camperlean.CONSUMER_COLOR.replace("Pink", "Rosa", inplace=True)
camperlean.CONSUMER_COLOR.replace("White", "Blanco", inplace=True)
#camperlean.CONSUMER_COLOR.replace("Nude", "Yellow", inplace=True)???
camperlean.CONSUMER_COLOR.replace("Yellow", "Amarillo", inplace=True)
camperlean.CONSUMER_COLOR.replace("Orange", "Naranja", inplace=True)
camperlean.CONSUMER_COLOR.replace("Purple", "Violeta", inplace=True)
camperlean.CONSUMER_COLOR.replace("Burgundy", "Burdeos", inplace=True)

# Unificación de CONSUMER_COLOR a un solo formato:
camperlean.CONSUMER_COLOR.replace('Marr?n', 'Marrón', inplace=True)

# Unificación de CREMALLERA:
camperlean.CREMALLERA.replace('NO', 'Sin cremallera', inplace=True)
camperlean.CREMALLERA.replace(np.nan, 'Sin cremallera', inplace=True)
camperlean.CREMALLERA.replace('SI', 'Con cremallera', inplace=True)
camperlean.CREMALLERA.replace('YES', 'Con cremallera', inplace=True)


# Unificación de CORDONES:
camperlean.CORDONES.replace('Without laces', 'Sin cordones', inplace=True)
camperlean.CORDONES.replace('With laces', 'Con cordones', inplace=True)

# Simplificado de OUTSOLE_SUELA_SUBTIPO:
camperlean.OUTSOLE_SUELA_SUBTIPO.replace('MEDIUM ( + 4.5 cm to 6 cm)', 'MEDIUM', inplace=True)
camperlean.OUTSOLE_SUELA_SUBTIPO.replace('HIGH (+ 6 cm)', 'HIGH', inplace=True)

# Unificación de PLANTILLA_EXTRAIBLE:
camperlean.PLANTILLA_EXTRAIBLE.replace('No extractable', 'No extraible', inplace=True)
camperlean.PLANTILLA_EXTRAIBLE.replace(np.nan, 'No extraible', inplace=True)
camperlean.PLANTILLA_EXTRAIBLE.replace('Extractable', 'Extraible', inplace=True)

# =========================================== FEATURE ENGINEERING =======================================

# Nueva variable CUSTOMER_ID_2 (conversión a numerico de CUSTOMER_ID):
camperlean['CUSTOMER_ID_2'] = label_encoder.fit_transform(camperlean['CUSTOMER_ID'])

# Extracción de temporada (F/W o S/S) de TEMPORADA_COMERCIAL_DESC y creación de nueva columna TEMPORADA:
camperlean['TEMPORADA'] = camperlean['TEMPORADA_COMERCIAL_DESC'].str.extract("(F/W|S/S)")[0]

# Extracción de año de TEMPORADA_COMERCIAL_DESC y creación de nueva columna ANO_TEMPORADA:
camperlean['ANO_TEMPORADA'] = camperlean.TEMPORADA_COMERCIAL_DESC.str[-4:]

# Nueva variable POPULARIDAD (Conteo de número de veces que aparece cada artículo por PRODUCTO_ID):
camperlean['POPULARIDAD'] = camperlean.groupby(['PRODUCTO_ID'])['PRODUCTO_ID'].transform('count')

# Nueva variable Categoría de Color:
camperlean['CAT_COLOR'] = camperlean.COLOR
camperlean.CAT_COLOR.replace(['Navy','Black', 'Dark Blue','Dark Gray','Charcoal'],"Dark colours",  inplace=True)
camperlean.CAT_COLOR.replace(['Medium Beige', 'Light Beige', 'Nude', 'Dark Beige'],"Nude colours",  inplace=True)
camperlean.CAT_COLOR.replace(['Dark Orange','Dark Yellow', 'Dark Pink', 'Dark Purple' ],"Winter Colourful",  inplace=True)
camperlean.CAT_COLOR.replace(['Brown','Medium Brown', 'Dark Brown','Rust/Copper' ] ,"Browns",  inplace=True)
camperlean.CAT_COLOR.replace('White Natural' ,"White",  inplace=True)
camperlean.CAT_COLOR.replace(['Lt. Pastel Grey','Lt/Pastel Yellow', 'Lt/Pastel Pink','Lt/Pastel Blue', 'Lt/Pastel Green','Lt/Pastel Purple', 'Lt/Pastel Orange'] ,"Pastel",  inplace=True)
camperlean.CAT_COLOR.replace(['Multi - Assorted','Medium Red','Medium Blue','Medium Yellow','Medium Green','Orange' ,'Yellow', 'Medium Orange','Turquiose Aqua', 'Medium Purple','Medium Pink'],"Colourful",  inplace=True)
camperlean.CAT_COLOR.replace(['Bright Yellow','Bright Orange', 'Bright Blue','Bright Pink', 'Bright Purple','Bright Green','Bright Red'] ,"Bright colours",  inplace=True)
camperlean.CAT_COLOR.replace([ 'Dark Green','Dark Red','Burgundy','Medium Gray'],"Neutral tones",  inplace=True)

# Nueva variable CAT_COLOR_2:
camperlean['CAT_COLOR_2'] = camperlean.CAT_COLOR
camperlean.CAT_COLOR_2.replace(['Colourful','Bright colours', 'Winter Colourful'], 'Colourful',  inplace=True)
camperlean.CAT_COLOR_2.replace(['Dark colours','Browns', 'Neutral tones'], 'Dark colours',  inplace=True)
camperlean.CAT_COLOR_2.replace(['White', 'Pastel', 'Nude colours'], 'Light colours',  inplace=True)

# Nueva variable CATEGORIA_2 (simplificado de CATEGORIA):
camperlean['CATEGORIA_2'] = camperlean['CATEGORIA']
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Shoe.*$)', 'Shoes')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Sneaker.*$)', 'Sneakers')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Boot.*$)', 'Boots')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*boot.*$)', 'Boots')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Moccasin.*$)', 'Moccasins')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Mocasin.*$)', 'Moccasins')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Slipper.*$)', 'Slippers')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Flip-Flop.*$)', 'Flip-Flops')
camperlean['CATEGORIA_2'] = camperlean.CATEGORIA_2.str.replace(r'(^.*Sandal.*$)', 'Sandals')


# ================================== SELECCIÓN DE COLUMNAS ÚTILES AL MODELO =============================

# Quitando columnas no necesarias (de momento)
camperlean = camperlean.drop(['FACTURA_POSICION_ID', #OJO DUPLICADOS
                              'ANO_MES_FACTURA',
                              'TEMPORADA_COMERCIAL_DESC',
                              'MATERIAL_ID',
                              'CANAL_VENTA_ID', 
                              'CANAL_VENTA_DESC',
                              'FACTURA_CLASE_DOCUMENTO_ID', 
                              'PAIS_CONTACTO_DESC', 
                              'NUMERO_DEUDOR_PAIS_DESC', 
                              'VENTA_DEVOLUCION', 
                              'CONTACTO_SN', 
                              'EDAD_SN', 
                              'JERARQUIA_1',
                              'CATEGORIA_INFERIDA'] , axis='columns')

# ========================================== REORDENADO DE COLUMNAS =======================================

# Reordenado de columnas para conservar coherencia del dataframe
camperlean = camperlean[['FACTURA_ID',
                         'CUSTOMER_ID',
                         'CUSTOMER_ID_2',
                         'ANO_FACTURA',
                         'MES_FACTURA',
                         'FECHA_FACTURA',
                         'IMP_VENTA_NETO_EUR',
                         'TEMPORADA_COMERCIAL_ID',
                         'TEMPORADA',
                         'ANO_TEMPORADA',
                         'PRODUCTO_ID',
                         'POPULARIDAD',
                         'TALLA',
                         'JERARQUIA_PROD_ID',
                         'GRUPO_ARTICULO_PRODUCTO_ID',
                         'GRUPO_ARTICULO',
                         'CONCEPTO',
                         'LINEA',
                         'GENERO_PRODUCTO',
                         'CATEGORIA',
                         'CATEGORIA_2',
                         'TIPOLOGIA',
                         'COLOR',
                         'CONSUMER_COLOR',
                         'CAT_COLOR',
                         'CAT_COLOR_2',
                         'CREMALLERA',
                         'CORDONES',
                         'OUTSOLE_SUELA_TIPO',
                         'OUTSOLE_SUELA_SUBTIPO',
                         #'PLANTILLA_EXTRAIBLE',
                         'GENERO_CONTACTO',
                         'EDAD_COMPRA',
                         'EDAD_RANGO_COMPRA',
                         'NUMERO_DEUDOR_PAIS_ID',
                         'PAIS_CONTACTO',
                         'CIUDAD_CONTACTO',
                         'IDIOMA_CONTACTO']]


/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launcher.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launcher.py:208: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launcher.py:209: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launcher.py:210: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launcher.py:211: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/darkxiii/.conda/envs/camper/lib/python3.7/site-packages/ipykernel_launche

# Clean Dataset

In [6]:
# Base de datos limpia:
camperlean.head()

,FACTURA_ID,CUSTOMER_ID,CUSTOMER_ID_2,ANO_FACTURA,MES_FACTURA,FECHA_FACTURA,IMP_VENTA_NETO_EUR,TEMPORADA_COMERCIAL_ID,TEMPORADA,ANO_TEMPORADA,PRODUCTO_ID,POPULARIDAD,TALLA,JERARQUIA_PROD_ID,GRUPO_ARTICULO_PRODUCTO_ID,GRUPO_ARTICULO,CONCEPTO,LINEA,GENERO_PRODUCTO,CATEGORIA,CATEGORIA_2,TIPOLOGIA,COLOR,CONSUMER_COLOR,CAT_COLOR,CAT_COLOR_2,CREMALLERA,CORDONES,OUTSOLE_SUELA_TIPO,OUTSOLE_SUELA_SUBTIPO,GENERO_CONTACTO,EDAD_COMPRA,EDAD_RANGO_COMPRA,NUMERO_DEUDOR_PAIS_ID,PAIS_CONTACTO,CIUDAD_CONTACTO,IDIOMA_CONTACTO
0,2856600006,heLvMOhLzUq8Q3JQDKC1VYitTQpF5BpDPxJmUK3eoaQ=,355134,2018,1,2018-01-01,155.75,85,F/W,2017,18637-001,72,42,101ATMATMWH18637,1.00,Zapatos Adulto,Atom,Atom Work,MEN,Men Shoe,Shoes,Blucher,Black,Negro,Dark colours,Dark colours,Sin cremallera,Con cordones,FLAT,FLAT,MAN,30.0,30-39,TR,TR,Izmit (Kocaeli),TR
1,2844600022,LF/nHsMbdxZfH9ymPjCNGMO7v5nwb4kLLoV77YZkqHA=,181053,2018,1,2018-01-01,112.07,85,F/W,2017,K100243-006,167,41,101TRNTRN0HK100243,1.00,Zapatos Adulto,Truman,Truman,MEN,Men Shoe,Shoes,Blucher,Black,Negro,Dark colours,Dark colours,Sin cremallera,Con cordones,FLAT,FLAT,UNDEFINED,42.0,40-49,US,US,Jersey City,EN
2,2844600022,LF/nHsMbdxZfH9ymPjCNGMO7v5nwb4kLLoV77YZkqHA=,181053,2018,1,2018-01-01,184.27,85,F/W,2017,16002-204,1097,41,101PELPELAH16002,1.00,Zapatos Adulto,Pelotas,Pelotas Ariel,MEN,Men Shoe,Shoes,Oxford,Dark Brown,Marrón,Browns,Dark colours,Sin cremallera,Con cordones,FLAT,FLAT,UNDEFINED,42.0,40-49,US,US,Jersey City,EN
3,2844600029,dZTljcECKzzhrFvudkxLjJGZOHMioStfcX351srHZrk=,323804,2018,1,2018-01-01,169.19,85,F/W,2017,46829-036,373,40,101PEUPEUPM46829,1.00,Zapatos Adulto,Peu,Peu Pista,WOMEN,Women Ankle Boot,Boots,Lace Up Bootie,Black,Negro,Dark colours,Dark colours,Sin cremallera,Con cordones,FLAT,FLAT,WOMAN,46.0,40-49,US,US,Mesa,EN
4,2844600014,Xb5/e2P6hITup7gs5k6G2jjFC/7Tahyp5V4txDKbzFU=,277513,2018,1,2018-01-01,132.55,85,F/W,2017,K300192-001,414,44,101PEUPEUCHK300192,1.00,Zapatos Adulto,Peu,Peu Cami,MEN,Men Ankle Boot,Boots,Basket bootie,Black,Negro,Dark colours,Dark colours,Con cremallera,Con cordones,FLAT,FLAT,UNDEFINED,49.0,40-49,US,US,Stafford,EN


# Saving new dataset

In [7]:
#camperlean.to_csv('CamperleanV.6.csv')

# Testing Results